In [9]:
import collections
import re
import glob
import tqdm
import codecs
import unidecode
from elasticsearch import Elasticsearch
import json

def save_to_json(path, lista):
    with open(path, 'w') as f:
        json.dump(lista, f, indent = 4)
        
def readFile(path, size=-1, clean=True):
    text = ''

    try:
        with codecs.open(path,'r', encoding='utf-8') as f:
            text = f.read(size)
    except:
        with codecs.open(path,'r', encoding='latin-1') as f:
            text = f.read(size)

    if clean:
        text = unidecode.unidecode(text)
        #text = remove_extra_spaces.sub(' ', text)
    return text

In [10]:
class EventSearcher(object):

    def __init__(self):
        self.events = collections.OrderedDict([
            ('__label__aviso_de_licitacao-tomada_de_precos', [r'TOMADA DE PRECOS']),            
            ('__label__pregao', [r'PREGAO']),
            ('__label__convite', [r'CARTA CONVITE', r'CONVITE - \d']),
            ('__label__credenciamento', [r'\sCREDENCIAMENTO']),
            ('__label__concorrencia', [r'\sCONCORRENCIA']),
            ('__label__pregaoeletronico', [r'PREGAO ELETRONICO']),
            ('__label__pregaopresencial', [r'PREGAO PRESENCIAL']),
            ('__label__leilao', [r'\sLEILAO']),
            ('__label__dialogo', [r'\sDIALOGO']),
            ('__label__extrato_de_contrato', [r'EXTRATO DE CONTRATO']),            
            ('__label__extrato_de_termo_aditivo', [r'TERMO ADITIVO']), 
            #CONTRATACAO DIRETA
            ('__label__extrato_de_dispensa_de_licitacao', [r'DISPENSA DE (LICITACAO|PROCESSO LICITATORIO)']),
            ('__label__extrato_de_inexigibilidade_de_licitacao ', [r'INEXIGIBILIDADE DE LICITACAO', r'INEXIGIBILIDADE']),            
            ('__label__contratacao_direta', [r'CONTRATACAO DIRETA']),
            ('__label__aviso_de_licitacao ', [r'AVISO DE LICITACAO']),            
            ('__label__aviso_de_licitacao-pregao', [r'\sLICITACAO'])            
        ])
        
        self.events_regex = {}
        
        for event in self.events.keys():
            self.events_regex[event] = []

            for regex in self.events[event]:
                regex = regex.replace(' ', r'\s*')
                self.events_regex[event].append(re.compile(regex, re.DOTALL | re.MULTILINE | re.IGNORECASE))
    
        
    def search(self, snippet):
        for event in self.events.keys():
            for regex in self.events_regex[event]:
                match = regex.search(snippet)
                if match is not None:
                    return event

        return None

In [11]:
maceio = glob.glob('/var/projects/tceal/materias/executivo/municipal/maceio/prefeitura/edicoes/2021/*/*/*.txt')
print(len(maceio))

ama = glob.glob('/var/projects/tceal/materias/executivo/municipal/ama/edicoes/2021/*/*/*.txt')
print(len(ama))

paths = maceio + ama
print(len(paths))

15078
31158
46236


In [12]:
ev = EventSearcher()

In [13]:
resultados = []
for path in tqdm.tqdm(paths):
    text = readFile(path)
    classe = ev.search(text)
    resultados.append({
        'tipo': classe,
        'path': path,
        'text': text
    })
    

100%|██████████| 46236/46236 [00:13<00:00, 3461.16it/s]


In [14]:
resultados = [r for r in resultados if r['tipo'] != None]
print(len(resultados))
#13852
#17122

21243


In [53]:
def get_setting():
    return {
              'analysis': {
                    'analyzer': {
                        'comma': {'pattern': ';', 'type': 'pattern'},
                        'br_analyzer': {
                            'filter': ['lowercase', 'asciifolding'],
                            'type': 'custom',
                            'tokenizer': 'standard'
                        }
                    }
                }
            }

In [54]:
def get_mapping():
    return {
        'properties': {
            'materia': {
                'properties': {                    
                    'path': {
                        'type': 'text',
                        'fields': {
                            'keyword': {
                                'type': 'keyword',
                                'ignore_above': 256
                            }
                        }
                    },
                    'tipo': {
                        'type': 'text',
                        'fields': {
                            'keyword': {
                                'type': 'keyword',
                                'ignore_above': 256
                            }
                        }
                    },
                    'snippet': {
                        'type': 'text',
                        'analyzer': 'br_analyzer'
                    },
                    'day': {
                        'type': 'text',
                        'fields': {
                            'keyword': {
                                'type': 'keyword',
                                'ignore_above': 256
                            }
                        }
                    },
                    'month': {
                        'type': 'text',
                        'fields': {
                            'keyword': {
                                'type': 'keyword',
                                'ignore_above': 256
                            }
                        }
                    },
                    'year': {
                        'type': 'text',
                        'fields': {
                            'keyword': {
                                'type': 'keyword',
                                'ignore_above': 256
                            }
                        }
                    },
                    'edition_number': {
                        'type': 'text',
                        'fields': {
                            'keyword': {
                                'type': 'keyword',
                                'ignore_above': 256
                            }
                        }
                    },
                    'edition_type': {
                        'type': 'text',
                        'fields': {
                            'keyword': {
                                'type': 'keyword',
                                'ignore_above': 256
                            }
                        }
                    },
                    'date': {
                        'type': 'date',
                        'format': 'yyyy-MM-dd'
                    }
                }
            }
        }
    }

In [55]:
settings = get_setting()
mappings = get_mapping()

config = {
    "settings": {
        "index": settings
    },
    "mappings" : mappings
}

In [42]:
es = Elasticsearch(['http://localhost:9200'], http_auth=('elastic', 'elastic@ais.mining'))
index_name = 'mineracao-maceio-ama-licitacoes-contratos'

In [45]:
es.indices.create(body=config, index=index_name)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'mineracao-maceio-ama-licitacoes-contratos'}

In [44]:
es.indices.delete(index=index_name)

{'acknowledged': True}

In [15]:
infos = json.load(open('municipios_materia.json'))

resultados_ = []
for result in tqdm.tqdm(resultados):
    info = infos[result['path']]
    
    info.update(result)
    
    #ret = es.index(index=index_name, body=info)
    resultados_.append(info)

100%|██████████| 21243/21243 [00:00<00:00, 459676.73it/s]


In [16]:
save_to_json('licitacoes_contratos3.json', resultados_)